## sparkmlhw01
采用Datasets操作，实现WordCount实例，并且按照count值降序显示前50行数据，其中word转换成小写，去除标点符号，去除停用词，考查点：  
1）  spark读取文件  
2）  dataset转换操作、聚合操作  
重点在数据清洗，转成小写，去除标点、停用词等，这里需要自己自定义停用词集合 和 标点符号集合  
3）  dataset排序及显示  

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
#生成SparkSession实例
spark = SparkSession.builder \
     .master("local[*]") \
     .appName("Word Count") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [11]:
df1 = spark.read.csv("file:///home/ian/code/data/sparkml/wordcount", sep='|', header=False)
type(df1)

pyspark.sql.dataframe.DataFrame

In [12]:
df1.show(5)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|  1|The Apache Hadoop...|
|  2|Apache Spark is a...|
|  3|Apache Storm is a...|
|  4|Below is a high-l...|
+---+--------------------+



In [15]:
df1 = df1.withColumnRenamed('_c0','index').withColumnRenamed('_c1','content')

In [16]:
df1.dtypes

[('index', 'string'), ('content', 'string')]

In [18]:
df2 = df1.select('content')

In [24]:
rdd1 = df2.rdd.flatMap(lambda x:x[0].split())\
    .map(lambda x:x.lower())\
    .filter(lambda x: x not in list(",.:!';"))\
    .filter(lambda x: x not in ["the","of","a","to","that","it","for","is","and"])\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda a,b:a+b)

In [30]:
df3 = spark.createDataFrame(rdd1)
df3.sort(df3['_2'].desc()).show()

+-----------+---+
|         _1| _2|
+-----------+---+
|     apache|  4|
|      storm|  3|
|      spark|  3|
|     hadoop|  2|
|    library|  2|
|distributed|  2|
|       data|  2|
|   designed|  2|
|computation|  2|
| processing|  2|
|    system.|  2|
|programming|  2|
|        sql|  2|
|       each|  2|
|         on|  2|
|    cluster|  2|
|         be|  2|
| high-level|  2|
|         in|  2|
|   supports|  2|
+-----------+---+
only showing top 20 rows

